

```
# Isto está formatado como código
```

# Santander Dev Week 2023 (ETL com Python)

**Contexto:** ETL baseado no dataset titanic.csv, onde farei algumas pesquisas e montarei um relatório.

**Condições do Problema:**

1. O dataset contém valores nulos, o que pode levar a falsos positivos em algumas entradas.
*   O dataset não possui informações sobre sobreviventes, faixa etária e classes sociais.

2. O objetivo será criar funções que resolvam esse problema.
3. Ao final, teremos o relatório com os dados extraídos..


## **E**xtract

1.   Importamos o pandas
2.   Carregamos o titanic.csv

In [ ]:
import pandas as pd

df = pd.read_csv('titanic.csv')

## **T**ransform

Tratando valores e definindos funções

In [ ]:
def definindo_genero(df):
  contagem_sexo = df['Sex'].value_counts()
  porcentagem_mulheres = (contagem_sexo['female'] /len(df)) *100
  porcentagem_homens = (contagem_sexo['male'] / len(df)) *100
  return f"Feminino: {porcentagem_mulheres:.2f}%\nMasculino: {porcentagem_homens:.2f}%"

In [ ]:
def definindo_classe_social(df):
    # Define os limites dos intervalos para cada classe social
    limites = [0, 30, 70, float('inf')]

    # Usamos o pd.cut() para calcular a contagem de cada classe social
    contagem_classe_social = pd.cut(df['Fare'], bins=limites, right=False).value_counts(sort=False)

    total = len(df)
    percentuais = (contagem_classe_social / total) * 100

    classes_sociais = ['Classe Baixa', 'Classe Média', 'Classe Alta']

    # Cria uma string vazia que será interada pela classes sociais e seus percentuais
    # a mensagem está no relatório mas também poderia ter sido definido abaixo.
    resultado = ""
    for classe, percentual in zip(classes_sociais, percentuais):
        resultado += f"\n{classe}: {percentual:.2f}%, "

    # Remove a vírgula extra no final e retorna a string
    return resultado[:-2]

In [ ]:
def classificar_faixa_etaria(idade):
    if pd.isna(idade):
        return 'Idade não confirmada'
    elif idade < 10:
        return 'Criança'
    elif idade < 14:
        return 'Pré-adolescente'
    elif idade < 18:
        return 'Adolescente'
    elif idade < 60:
        return 'Adulto'
    else:
        return 'Idoso'

In [ ]:
def criar_coluna_porcentagem_faixa_etaria():
    df['Faixa_Etaria'] = df['Age'].apply(classificar_faixa_etaria)

    contagem_faixa_etaria = df['Faixa_Etaria'].value_counts()

    total_passageiros = len(df)
    porcentagens_faixa_etaria = (contagem_faixa_etaria / total_passageiros) * 100

    porcentagens_formatadas = porcentagens_faixa_etaria.apply(lambda x: f'{x:.2f}%')

    return porcentagens_formatadas

In [ ]:
# Essa função achama a função classificar_faixa_etaria(idade)
def calcular_e_imprimir_contagem_faixa_etaria(df):

    df['Faixa_Etaria'] = df['Age'].apply(classificar_faixa_etaria)

    contagem_faixa_etaria = df['Faixa_Etaria'].value_counts()
    contagem_faixa_etaria_df = pd.DataFrame(contagem_faixa_etaria)

    return contagem_faixa_etaria_df

In [ ]:
def percentual_geral_sobreviventes():
  total_passageiros = len(df)
  # 1 = vivo, 0 = falecido
  total_sobreviventes = len(df[df['Survived'] == 1])
  total_nao_sobreviventes = len(df[df['Survived'] == 0])
  percentual_sobreviventes = (total_sobreviventes / total_passageiros) * 100
  percentual_nao_sobreviventes = (total_nao_sobreviventes / total_passageiros) * 100

  return (f"Percentual de pessoas que sobreviveram: {percentual_sobreviventes:.2f}%\n" +
            f"Percentual de pessoas que morreram: {percentual_nao_sobreviventes:.2f}%")

## **L**oad

Criando um relatório sobre o  desastre do titanic

In [ ]:
# Mesmo o dataset sendo pequeno, não preferi salvar em variáveis as funções, pois no momento atual não há previsão
# de reuso
relatorio = f"""
O Titanic era composto por passageiros do gênero:
{definindo_genero(df)}

A classe social no Titanic era dividida em:
{definindo_classe_social(df)}

A divisão por faixa etária era de:
{calcular_e_imprimir_contagem_faixa_etaria(df)}

Número de sobreviventes por faixa etária:
{df[df['Survived'] == 1]['Faixa_Etaria'].value_counts().to_string()}

O número de falecidos por faixa etária:
{df[df['Survived'] == 0]['Faixa_Etaria'].value_counts().to_string()}

Percentual geral de sobreviventes e falecidos do Titanic:
{percentual_geral_sobreviventes()}
"""

print(relatorio)


O Titanic era composto por passageiros do gênero:
Feminino: 36.36%
Masculino: 63.64%

A classe social no Titanic era dividida em:

Classe Baixa: 73.44%, 
Classe Média: 14.11%, 
Classe Alta: 12.20%

A divisão por faixa etária era de:
                      Faixa_Etaria
Adulto                         277
Idade não confirmada            86
Criança                         20
Idoso                           14
Adolescente                     13
Pré-adolescente                  8

Número de sobreviventes por faixa etária:
Adulto                  103
Idade não confirmada     25
Criança                   9
Idoso                     7
Adolescente               5
Pré-adolescente           3

O número de falecidos por faixa etária:
Adulto                  174
Idade não confirmada     61
Criança                  11
Adolescente               8
Idoso                     7
Pré-adolescente           5

Percentual geral de sobreviventes e falecidos do Titanic:
Percentual de pessoas que sobreviveram: 36